#Required packages



In [10]:
!pip install torch transformers kfp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.1/269.1 kB 11.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 23.9 MB/s eta 0:00:00
  Created wheel for kfp: filename=kfp-2.13.0-py3-none-any.whl size=366775 sha256=554d9e3ab827bf8d421f35eb4ec827475504383a5d637311275590d679ba932d
  Stored in directory: /root/.cache/pip/wheels/d0/10/51/5eca8bddd084ce32a64771f8ebb11c4ba6368ea506e2f282d1
  Created wheel for kfp-server-api: filename=kfp_server_api-2.4.0-py3-none-any.whl size=116527 sha256=efa57f24b5e1e2bfbb56ab94aeb9d0e46f1a093ede7bfaf1b5796fec3b1bb0f6
  Stored in directory: /root/.cache/pip/wheels/8b/21/cc/58400528e14863ac19175d355d3392384d8ba63bc9d6e219f4
Successfully built kfp kfp-server-api
  Attempting uninstall: protobuf
 

In [11]:
!pip install accelerate

# STEP 1: Mount Google Drive


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#KFP generation using Qwen2.5-Coder-0.5B

In [12]:
# STEP 1: Imports
import json
import logging
from pathlib import Path
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import BitsAndBytesConfig  # For quantization (optional)

# STEP 2: Setup logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('kfp_generation.log'),  # Save logs to file
        logging.StreamHandler()  # Also print to console
    ]
)
logger = logging.getLogger(__name__)

# STEP 3: Configuration
MODEL_PATH = "Qwen/Qwen2.5-Coder-1.5B-Instruct"  # Instruction-tuned model
PROMPT_FILE = "prompts.json"
OUTPUT_DIR = Path("kfp_eval_samples")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# STEP 4: Install dependencies (optional, run in terminal if needed)
"""
pip install torch transformers kfp
"""

# STEP 5: Load prompts
try:
    with open(PROMPT_FILE, encoding="utf-8") as f:
        prompts = json.load(f)
    logger.info(f"Loaded {len(prompts)} prompts from {PROMPT_FILE}")
except FileNotFoundError:
    logger.error(f"{PROMPT_FILE} not found. Please ensure it exists.")
    exit(1)
except json.JSONDecodeError:
    logger.error(f"{PROMPT_FILE} contains invalid JSON.")
    exit(1)

# STEP 6: Load model and tokenizer
logger.info(f"Downloading and loading model from {MODEL_PATH}...")
try:
    # Check device availability
    device = "cuda" if torch.cuda.is_available() else "cpu"
    logger.info(f"Using device: {device}")
    if device == "cuda":
        logger.info(f"GPU: {torch.cuda.get_device_name(0)}, VRAM: {torch.cuda.memory_allocated(0)/1e9:.2f} GB")

    # Optional: 4-bit quantization for low VRAM (uncomment if needed)
    """
    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_quant_type="nf4"
    )
    """

    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
    logger.info("Tokenizer loaded successfully.")

    # Load model
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_PATH,
        torch_dtype=torch.float16,  # FP16 for efficiency
        device_map="auto",  # Auto-distribute across GPU/CPU
        # quantization_config=quantization_config  # Uncomment for quantization
    )
    logger.info("Model loaded successfully.")
except Exception as e:
    logger.error(f"Error loading model or tokenizer: {e}")
    exit(1)

# STEP 7: Generator function
def generate_kfp_code(prompt, file_name):
    messages = [
        {
            "role": "system",
            "content": (
                "You are a Kubeflow Pipelines expert. Generate valid KFP Python code that compiles with the Kubeflow Pipelines SDK. "
                "Use the `@dsl.pipeline` decorator with the name specified in the user prompt. "
                "Define components using `@component` decorators with appropriate inputs and outputs (e.g., Artifact, Dataset, Model). "
                "Use `kfp` and `kfp.dsl` libraries, include necessary imports, and ensure syntactic correctness. "
                "Do not include explanations, comments, or non-code content."
            )
        },
        {
            "role": "user",
            "content": (
                f"{prompt}\n\n"
                f"Generate the Kubeflow Pipeline code for the file named `{file_name}`. "
                f"Use `{file_name}` as the name of the `@dsl.pipeline` function to match the original file. "
                f"Ensure the code is complete, syntactically correct, and adheres to the Kubeflow Pipelines SDK."
            )
        }
    ]
    try:
        chat_text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        inputs = tokenizer(chat_text, return_tensors="pt").to(model.device)

        generated_ids = model.generate(
            **inputs,
            max_new_tokens=4096,  # Sufficient for complex pipelines
            do_sample=False,  # Deterministic output
            temperature=0.2,  # Low for precise code
            top_p=0.9  # Control output diversity
        )

        output_ids = generated_ids[0][inputs.input_ids.shape[-1]:]
        response = tokenizer.decode(output_ids, skip_special_tokens=True)

        # Check for prompt repetition
        if response.strip().startswith("Here is the Kubeflow Pipeline named"):
            logger.warning(f"Generated output for {file_name} appears to be the prompt.")
            return f"# Error: Generated output for {file_name} is the prompt, not code."

        return response
    except Exception as e:
        logger.error(f"Generation failed for {file_name}: {e}")
        return f"# Error: Qwen2.5 generation failed for {file_name}: {e}"

# STEP 8: Test model with a simple KFP prompt
logger.info("Running test generation to verify model...")
test_prompt = (
    "Define a KFP pipeline named `test_pipeline` with one component: "
    "`print_message` takes a string `message` and prints it, returning the same string. "
    "The pipeline takes a string input `message` and returns the component's output."
)
test_file_name = "test_pipeline"
test_code = generate_kfp_code(test_prompt, test_file_name)

# Save test output
test_file_path = OUTPUT_DIR / f"{test_file_name}.py"
try:
    test_file_path.write_text(test_code, encoding="utf-8")
    logger.info(f"Saved test KFP code to {test_file_path}")
except Exception as e:
    logger.error(f"Error writing test output: {e}")

# Log test output
logger.info(f"Test output for {test_file_name}:\n{test_code}")

# STEP 9: Generate KFP code for all prompts
logger.info("Generating KFP code for all prompts in prompts.json...")
for sample_id, item in enumerate(prompts, 1):
    prompt = item["structured_prompt"]
    repo = item["repo"].replace("/", "__")
    file_name = Path(item["file"]).stem
    pipeline_name = f"{repo}__{file_name}"

    logger.info(f"Generating KFP code for sample {sample_id}: {pipeline_name}")
    code = generate_kfp_code(prompt, file_name)

    # Save to file
    file_path = OUTPUT_DIR / f"{pipeline_name}.py"
    try:
        file_path.write_text(code, encoding="utf-8")
        logger.info(f"Saved KFP code to {file_path}")
    except Exception as e:
        logger.error(f"Error writing to {file_path}: {e}")

logger.info("✅ KFP code generation completed.")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p'